# Parameters and Searching 

## Build a Basic Decision Tree Model for Predicting Spam

1. Download the spam.csv file
2. Load the data into a dataframe pandas
3. Process by dealing with missing values and non-numerical values
4. Split into target and features
5. Split into training and testing data
6. Train model on the training data
7. Find score on the testing data


## Adjusting Parameters

Now we talk about some different parameters a Decision Tree classifier can have, and we write a function that tells us how score changes with those parameters. 

## We do the same thing, but with GridSearchCV

## Random Forests

Now we introduce Random Forests, which are like groups of Decision Trees trained on subsets of the data. 

## Ensemble Classifiers 